# Linear Regression

TODO: brief intro

- MSE loss (what is a loss function?)
- math formula of the rlt 


## brief intro to scikit-learn (link)

TODO

## Simple linear regression

TODO: math formula

In [ ]:
# TODO: use synthetic data x-y from data/xxx.csv to do a linear regression

# read csv file

# use np or pd to store the data (review the preview workshop x)

# use linear regression method in scikit-learn to do linear regression (link)

# plot the rlt by using matplotlib.pyplot (review the preview workshop x)

Think: what do you notice from the plot?

## Multiple linear regression

TODO: math formula

In [ ]:
# TODO: use synthetic data x-y from data/xxx.csv to do a linear regression

# read csv file

# use np or pd to store the data (review the preview workshop x)

# use linear regression method in scikit-learn to do linear regression (link)


## Polynomial regression

Overfitting

Underfitting

Choose proper model

In [ ]:
# TODO: use synthetic data x-y from data/xxx.csv to do a linear regression

# read csv file

# use np or pd to store the data (review the preview workshop x)

# use linear regression method in scikit-learn to do linear regression (link)

# plot the rlt by using matplotlib.pyplot (review the preview workshop x)

## Exercise: a real task

by using a real dataset for prediction, randomly divide dataset into 80% train and 20% test

test accuracy on the test set to see what our linear regression can do

- data preprocessing (data cleaning, normalization, etc.)


In [ ]:
# TODO: find a proper dataset (can be directly imported by a line of code)

# preprocessing

# using method in scikit-learn to do the fitting

# test accuracy


Think: do you think the accuracy of this task is high or low, why?

# Logistic regression

TODO: intro

using images when needed

In [ ]:
# TODO: make a logistic regression task, using either synthetic data or find a dataset

# plot the rlt if possible

Think: what's the strengths and weaknesses of logistic regression (compared to the linear regression)?


Think: When shall we use the linear regression, when shall we use the logistic regression?

# GLM

TODO: intro

- what hyperparams does it have and what's the meaning of them?

- how can we tune the hyperparams to get a better rlt?

(I'm not familiar with GLM so not sure what to write now)

# Exercise: Using GLM for xx prediction

TODO: find a proper dataset and make this exercise

# Wrap up

brief conclusion of this workshop